# Usecase: Nomination of compound combinations from KG.

This notebook demonstrates the capability of BioDataFuse to nominate compound combinations.

In [1]:
# Import modules
import pickle

import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd

from pyBiodatafuse import id_mapper
from pyBiodatafuse.annotators import (
    disgenet,
    minerva,
    molmedb,
    opentargets,
    pubchem,
    stringdb,
    wikipathways,
)
from pyBiodatafuse.graph import generator
from pyBiodatafuse.utils import combine_sources

### List of compounds for building the network

In [7]:
data_input = pd.read_csv("compounds.tsv", sep="\t", dtype=str)
data_input.head()

,cid,chembl_id,smiles,InChI Key
0,1697,CHEMBL268868,O=C1NC(=O)c2cc(Nc3ccccc3)c(Nc3ccccc3)cc21,AAALVYBICLMAMA-UHFFFAOYSA-N
1,46943432,CHEMBL1232461,CCNC(=O)C[C@@H]1N=C(c2ccc(Cl)cc2)c2cc(OC)ccc2-...,AAAQFGUYHFJNHI-SFHVURJKSA-N
2,59620153,CHEMBL4303369,COc1ccc2[nH]c3c(-c4cccc5ccccc45)nccc3c2c1,AABFWJDLCCDJJN-UHFFFAOYSA-N
3,57267,CHEMBL8946,CCc1cc(Cl)c(OC)c(C(=O)NC[C@@H]2CCCN2CC)c1O,AADCDMQTJNYOSS-LBPRGKRZSA-N
4,15942827,CHEMBL3137322,O[C@H]([C@H](O)[C@@H]1CO1)[C@H]1CO1,AAFJXZWCNVJTMK-GUCUJZIJSA-N


In [8]:
f"Number of Compounds: {len(data_input)}"

'Number of Compounds: 5635'

### Entity resolution for compounds

**NOTE:** Bridgebd xrefs requires the identifiers to be list of strings.

In [9]:
df = data_input[["cid"]]
df.rename(columns={"cid": "identifier"}, inplace=True)

In [10]:
bridgdb_df, bridgdb_metadata = id_mapper.bridgedb_xref(
    identifiers=df,
    input_species="Human",
    input_datasource="PubChem Compound",  # ChEMBL compound
    output_datasource="All",
)
bridgdb_df.head()

,identifier,identifier.source,target,target.source
0,1697,PubChem-compound,145915-58-8,CAS
1,1697,PubChem-compound,AAALVYBICLMAMA-UHFFFAOYSA-N,InChIKey
2,1697,PubChem-compound,53110,ChEBI
3,1697,PubChem-compound,CHEMBL268868,ChEMBL compound
4,1697,PubChem-compound,CHEBI:53110,ChEBI


In [11]:
molmedb_transporter_inhibited_df, molmedb_transporter_inhibited_metadata = (
    molmedb.get_compound_gene_inhibitor(bridgedb_df=bridgdb_df)
)
molmedb_transporter_inhibited_df

KeyError: 'inhibitorInChIKey'

In [ ]:
def building_graph():
    # Compound - gene interactions
    molmedb_transporter_inhibited_df, molmedb_transporter_inhibited_metadata = (
        molmedb.get_compound_gene_inhibitor(bridgedb_df=bridgdb_df)
    )

### Combing all the results into single dataframe

In [30]:
combined_df = combine_sources(
    [
        minerva_df,
        wikipathways_df,
        opentargets_reactome_df,
        opentargets_go_df,
        disgenet_df,
        disease_df,
        opentargets_drug_df,
        inhibitor_df,
        string_ppi_metadata,
    ]
)

In [31]:
combined_df.head(4)

,identifier,identifier.source,target,target.source,Bgee,MINERVA,WikiPathways,OpenTargets_Reactome,OpenTargets_GO,OpenTargets_Location,OpenTargets_Diseases,OpenTargets_Compounds,MolMeDB_transporter_inhibitor,PubChem_Assays,StringDB
0,AHR,HGNC,ENSG00000106546,Ensembl,"[{'anatomical_entity_id': 'UBERON_0000178', 'a...","[{'pathway_id': 953.0, 'pathway_label': 'Kynur...","[{'pathway_id': 'WP5044', 'pathway_label': 'Ky...","[{'pathway_label': 'Endogenous sterols', 'path...","[{'go_id': 'GO:0005667', 'go_name': 'transcrip...","[{'location_id': 'SL-0086', 'location': 'Cytop...","[{'disease_id': 'umls:C0033860', 'disease_name...","[{'chembl_id': 'CHEMBL259571', 'drugbank_id': ...","[{'compound_name': nan, 'InChIKey': nan, 'SMIL...","[{'pubchem_assay_id': nan, 'assay_type': nan, ...","[{'stringdb_link_to': 'SCN4A', 'Ensembl': 'ENS..."
1,CHRNG,HGNC,ENSG00000196811,Ensembl,"[{'anatomical_entity_id': 'UBERON_0000178', 'a...","[{'pathway_id': nan, 'pathway_label': nan, 'pa...","[{'pathway_id': nan, 'pathway_label': nan, 'pa...",[{'pathway_label': 'Highly sodium permeable po...,"[{'go_id': 'GO:0015464', 'go_name': 'acetylcho...","[{'location_id': 'SL-0219', 'location': 'Posts...","[{'disease_id': 'umls:C0085631', 'disease_name...","[{'chembl_id': 'CHEMBL1200641', 'drugbank_id':...","[{'compound_name': nan, 'InChIKey': nan, 'SMIL...","[{'pubchem_assay_id': nan, 'assay_type': nan, ...","[{'stringdb_link_to': 'SCN4A', 'Ensembl': 'ENS..."
2,DMD,HGNC,ENSG00000198947,Ensembl,"[{'anatomical_entity_id': 'UBERON_0000178', 'a...","[{'pathway_id': nan, 'pathway_label': nan, 'pa...","[{'pathway_id': 'WP2858', 'pathway_label': 'Ec...",[{'pathway_label': 'Striated Muscle Contractio...,"[{'go_id': 'GO:0016010', 'go_name': 'dystrophi...","[{'location_id': 'SL-0039', 'location': 'Cell ...","[{'disease_id': 'umls:C0013264', 'disease_name...","[{'chembl_id': 'CHEMBL2108278', 'drugbank_id':...","[{'compound_name': nan, 'InChIKey': nan, 'SMIL...","[{'pubchem_assay_id': nan, 'assay_type': nan, ...",[]
3,HTR3A,HGNC,ENSG00000166736,Ensembl,"[{'anatomical_entity_id': 'UBERON_0000178', 'a...","[{'pathway_id': nan, 'pathway_label': nan, 'pa...","[{'pathway_id': 'WP706', 'pathway_label': 'Sud...",[{'pathway_label': 'Neurotransmitter receptors...,"[{'go_id': 'GO:1904602', 'go_name': 'serotonin...","[{'location_id': 'SL-0219', 'location': 'Posts...","[{'disease_id': 'EFO_0003843', 'disease_name':...","[{'chembl_id': 'CHEMBL56564', 'drugbank_id': '...","[{'compound_name': nan, 'InChIKey': nan, 'SMIL...","[{'pubchem_assay_id': nan, 'assay_type': nan, ...","[{'stringdb_link_to': 'DMD', 'Ensembl': 'ENSP0..."


In [32]:
combined_df.shape

(50, 15)

### Exporting the database in pickle format

In [33]:
with open("combined_df.pkl", "wb") as out:
    pickle.dump(combined_df, out)

## Creating a graph from the annotated dataframe

In [34]:
fuse_df = generator.load_dataframe_from_pickle("combined_df.pkl")

In [35]:
fuse_df.head()

,identifier,identifier.source,target,target.source,Bgee,MINERVA,WikiPathways,OpenTargets_Reactome,OpenTargets_GO,OpenTargets_Location,OpenTargets_Diseases,OpenTargets_Compounds,MolMeDB_transporter_inhibitor,PubChem_Assays,StringDB
0,AHR,HGNC,ENSG00000106546,Ensembl,"[{'anatomical_entity_id': 'UBERON_0000178', 'a...","[{'pathway_id': 953.0, 'pathway_label': 'Kynur...","[{'pathway_id': 'WP5044', 'pathway_label': 'Ky...","[{'pathway_label': 'Endogenous sterols', 'path...","[{'go_id': 'GO:0005667', 'go_name': 'transcrip...","[{'location_id': 'SL-0086', 'location': 'Cytop...","[{'disease_id': 'umls:C0033860', 'disease_name...","[{'chembl_id': 'CHEMBL259571', 'drugbank_id': ...","[{'compound_name': nan, 'InChIKey': nan, 'SMIL...","[{'pubchem_assay_id': nan, 'assay_type': nan, ...","[{'stringdb_link_to': 'SCN4A', 'Ensembl': 'ENS..."
1,CHRNG,HGNC,ENSG00000196811,Ensembl,"[{'anatomical_entity_id': 'UBERON_0000178', 'a...","[{'pathway_id': nan, 'pathway_label': nan, 'pa...","[{'pathway_id': nan, 'pathway_label': nan, 'pa...",[{'pathway_label': 'Highly sodium permeable po...,"[{'go_id': 'GO:0015464', 'go_name': 'acetylcho...","[{'location_id': 'SL-0219', 'location': 'Posts...","[{'disease_id': 'umls:C0085631', 'disease_name...","[{'chembl_id': 'CHEMBL1200641', 'drugbank_id':...","[{'compound_name': nan, 'InChIKey': nan, 'SMIL...","[{'pubchem_assay_id': nan, 'assay_type': nan, ...","[{'stringdb_link_to': 'SCN4A', 'Ensembl': 'ENS..."
2,DMD,HGNC,ENSG00000198947,Ensembl,"[{'anatomical_entity_id': 'UBERON_0000178', 'a...","[{'pathway_id': nan, 'pathway_label': nan, 'pa...","[{'pathway_id': 'WP2858', 'pathway_label': 'Ec...",[{'pathway_label': 'Striated Muscle Contractio...,"[{'go_id': 'GO:0016010', 'go_name': 'dystrophi...","[{'location_id': 'SL-0039', 'location': 'Cell ...","[{'disease_id': 'umls:C0013264', 'disease_name...","[{'chembl_id': 'CHEMBL2108278', 'drugbank_id':...","[{'compound_name': nan, 'InChIKey': nan, 'SMIL...","[{'pubchem_assay_id': nan, 'assay_type': nan, ...",[]
3,HTR3A,HGNC,ENSG00000166736,Ensembl,"[{'anatomical_entity_id': 'UBERON_0000178', 'a...","[{'pathway_id': nan, 'pathway_label': nan, 'pa...","[{'pathway_id': 'WP706', 'pathway_label': 'Sud...",[{'pathway_label': 'Neurotransmitter receptors...,"[{'go_id': 'GO:1904602', 'go_name': 'serotonin...","[{'location_id': 'SL-0219', 'location': 'Posts...","[{'disease_id': 'EFO_0003843', 'disease_name':...","[{'chembl_id': 'CHEMBL56564', 'drugbank_id': '...","[{'compound_name': nan, 'InChIKey': nan, 'SMIL...","[{'pubchem_assay_id': nan, 'assay_type': nan, ...","[{'stringdb_link_to': 'DMD', 'Ensembl': 'ENSP0..."
4,SCN4A,HGNC,ENSG00000007314,Ensembl,"[{'anatomical_entity_id': 'UBERON_0000178', 'a...","[{'pathway_id': nan, 'pathway_label': nan, 'pa...","[{'pathway_id': nan, 'pathway_label': nan, 'pa...",[{'pathway_label': 'Phase 0 - rapid depolarisa...,"[{'go_id': 'GO:0035725', 'go_name': 'sodium io...","[{'location_id': 'SL-0039', 'location': 'Cell ...","[{'disease_id': 'EFO_0000432', 'disease_name':...","[{'chembl_id': 'CHEMBL1077896', 'drugbank_id':...","[{'compound_name': '3-phenyl-1h-pyrazole', 'In...","[{'pubchem_assay_id': nan, 'assay_type': nan, ...",[]


In [36]:
pygraph = generator.networkx_graph(fuse_df)

### Store the graph

In [37]:
with open("networkx_graph.pkl", "wb") as out:
    pickle.dump(pygraph, out)

## Visualize the graph

In [38]:
# pos = nx.circular_layout(pygraph)

# plt.figure(3, figsize=(30, 30))
# nx.draw(pygraph, pos)
# plt.show()

In [ ]:
from pyBiodatafuse.graph import cytoscape, neo4j

neo4j.save_graph_to_graphml(pygraph, output_path="graph_to-test.graphml")
cytoscape.load_graph(pygraph, network_name="test")